In [1]:
!pip install xmltodict
import pandas as pd
import kagglehub
import xmltodict, pandas as pd

## Loading CSV files & EDA & Proofs

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)

path_videogamesales = kagglehub.dataset_download("gregorut/videogamesales")
print("Path to dataset files:", path_videogamesales)
path_metacritic = kagglehub.dataset_download("mohammedali10/metacritic-games-of-all-time")
print("Path to dataset files:", path_metacritic)
path_playtime = kagglehub.dataset_download("b4n4n4p0wer/how-long-to-beat-video-game-playtime-dataset")
print("Path to dataset files:", path_playtime)

Using Colab cache for faster access to the 'videogamesales' dataset.
Path to dataset files: /kaggle/input/videogamesales
Using Colab cache for faster access to the 'metacritic-games-of-all-time' dataset.
Path to dataset files: /kaggle/input/metacritic-games-of-all-time
Using Colab cache for faster access to the 'how-long-to-beat-video-game-playtime-dataset' dataset.
Path to dataset files: /kaggle/input/how-long-to-beat-video-game-playtime-dataset


In [ ]:
df_videogamesales = pd.read_csv(f"{path_videogamesales}/vgsales.csv").drop("Rank",axis=1)
df_metacritic = pd.read_csv(f"{path_metacritic}/Metacritic_games_of_all_time.csv",index_col=0)
df_playtime = pd.read_csv(f"{path_playtime}/hltb_dataset_filtered.csv")
df_playtime=df_playtime[df_playtime["type"]=="game"]
df_playtime=df_playtime[["name","platform","genres","developer","publisher","release_date","main_story","main_plus_sides","completionist"]]
df_videogamesales["Year"]= df_videogamesales["Year"].astype("Int32")
df_metacritic["release"] = pd.to_datetime(df_metacritic["release"], errors='coerce').dt.year
df_metacritic["genre(s)"] = (df_metacritic["genre(s)"].fillna("").apply(lambda s: [g.strip() for g in s.split(",")] if isinstance(s, str) else s))
df_metacritic['user_score'] = pd.to_numeric(df_metacritic['user_score'], errors='coerce')
df_playtime["release_date"] = pd.to_datetime(df_playtime["release_date"], errors='coerce').dt.year.astype('Int32')
df_playtime["genres"] = (df_playtime["genres"].fillna("").apply(lambda s: [g.strip() for g in s.split(",")] if isinstance(s, str) else s))
#### This is a drastic change
df_playtime.platform = df_playtime['platform'].str.split(',')
df_playtime = df_playtime.explode('platform')
df_playtime['platform'] = df_playtime['platform'].str.strip()
df_playtime.reset_index(drop=True, inplace=True)

display(df_videogamesales.head(2))
print(df_videogamesales.shape)
print("-"*200)
display(df_metacritic.head(2))
print(df_metacritic.shape)
print("-"*200)
display(df_playtime.head(2))
print(df_playtime.shape)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24


(16598, 10)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,title,release,platform,critic_score,user_score,developer,genre(s),rating
1,Portal 2,2011,Xbox 360,95.0,8.8,Valve Software,"[Action, Shooter, Shooter, First-Person, Sci-Fi, Sci-Fi, Arcade]",E10+
2,Metal Gear Solid V: The Phantom Pain,2015,Xbox One,95.0,7.5,Konami,"[Modern, Modern, Action Adventure, Open-World]",M


(14666, 8)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


,name,platform,genres,developer,publisher,release_date,main_story,main_plus_sides,completionist
0,688(I) Hunter/Killer,PC,[Simulation],Sonalysts,Electronic Arts,1997,10.62,35.37,15.83
1,'Splosion Man,Xbox 360,[Action],Twisted Pixel Games,Microsoft Games Studios,2009,7.60,9.23,18.77


(97231, 9)


In [ ]:
display(df_videogamesales.Platform.dropna().unique())
display(df_metacritic.platform.dropna().unique())
display(df_playtime.platform.dropna().unique())

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

array(['Xbox 360', 'Xbox One', 'PC', 'PlayStation 4', 'PlayStation 3',
       'PlayStation 2', '3DS', 'Wii U', 'Switch', 'Xbox Series X',
       'PlayStation 5', 'PlayStation Vita', 'Stadia'], dtype=object)

array(['PC', 'Xbox 360', 'Nintendo Switch', 'PlayStation 2',
       'PlayStation 4', 'PlayStation Portable', 'PlayStation 3', 'Wii U',
       'PlayStation', 'Amiga', 'Amstrad CPC', 'Atari ST', 'BBC Micro',
       'Commodore 64', 'MSX', 'ZX Spectrum', 'Nintendo 3DS',
       'Nintendo DS', 'Arcade', 'NES', 'Mobile', 'Wii', 'Dreamcast',
       'Game Boy Color', 'Nintendo 64', 'Nintendo GameCube', 'Browser',
       'FM-7', 'NEC PC-88', 'Sharp X1', 'TurboGrafx-16', 'Xbox',
       'PlayStation Vita', 'Xbox One', 'Mac', 'Atari 7800',
       'Atari 8-bit Family', 'Sega Master System', 'Atari 2600',
       'Super Nintendo', 'Sega Mega Drive/Genesis', 'Game Boy Advance',
       '3DO', 'PlayStation 5', 'Game Boy', 'Neo Geo', 'Neo Geo CD',
       'Xbox Series X/S', 'Sega 32X', 'FM Towns', 'Sega Saturn',
       'Sega CD', 'Ouya', 'Apple II', 'Linux', 'Sega Game Gear',
       'Atari Jaguar', 'NEC PC-98', 'TurboGrafx-CD', 'ColecoVision',
       'N-Gage', 'OnLive', 'Atari 5200', 'SG-1000', 'Neo Geo Po

In [ ]:
p_sales      = set(df_videogamesales.Platform.dropna().unique())
p_meta       = set(df_metacritic.platform.dropna().unique())
p_playtime   = set(df_playtime.platform.dropna().unique())

appears_in_2plus = (
    (p_sales & p_meta) |
    (p_sales & p_playtime) |
    (p_meta  & p_playtime)
)

print("Counts:",
      "\n- sales:", len(p_sales),
      "\n- metacritic:", len(p_meta),
      "\n- playtime:", len(p_playtime),
      "\n- in ≥2 datasets:", len(appears_in_2plus))


Counts: 
- sales: 31 
- metacritic: 13 
- playtime: 102 
- in ≥2 datasets: 13


In [ ]:
platform_map_overlapping = {
    # Xbox family
    "X360": "Xbox 360",
    "XOne": "Xbox One",

    # PlayStation family
    "PS4": "PlayStation 4",
    "PS3": "PlayStation 3",
    "PS2": "PlayStation 2",
    "PSV": "PlayStation Vita",

    # Nintendo family
    "Wii": "Wii",      # Keeping it for safety
    "DS": "Nintendo DS",
    "3DS": "Nintendo 3DS",
    "GC": "Nintendo GameCube",
    "GBA": "Game Boy Advance",
    "NES": "NES",
    "SNES": "Super Nintendo",

    # PC
    "PC": "PC",
    # (If your other datasets use "PC" consistently, this is fine.)
}


In [ ]:
platform_map_overlapping.items()

dict_items([('X360', 'Xbox 360'), ('XOne', 'Xbox One'), ('PS4', 'PlayStation 4'), ('PS3', 'PlayStation 3'), ('PS2', 'PlayStation 2'), ('PSV', 'PlayStation Vita'), ('Wii', 'Wii'), ('DS', 'Nintendo DS'), ('3DS', 'Nintendo 3DS'), ('GC', 'Nintendo GameCube'), ('GBA', 'Game Boy Advance'), ('NES', 'NES'), ('SNES', 'Super Nintendo'), ('PC', 'PC')])

In [ ]:
# keep only keys that appear in sales AND the mapped-to value appears in the 2+ set
filtered_map = {
    k: v for k, v in platform_map_overlapping.items()
    if (k in p_sales)
}
filtered_map


{'X360': 'Xbox 360',
 'XOne': 'Xbox One',
 'PS4': 'PlayStation 4',
 'PS3': 'PlayStation 3',
 'PS2': 'PlayStation 2',
 'PSV': 'PlayStation Vita',
 'Wii': 'Wii',
 'DS': 'Nintendo DS',
 '3DS': 'Nintendo 3DS',
 'GC': 'Nintendo GameCube',
 'GBA': 'Game Boy Advance',
 'NES': 'NES',
 'SNES': 'Super Nintendo',
 'PC': 'PC'}

In [ ]:
df_videogamesales["Platform_norm"] = (
    df_videogamesales["Platform"]
      .replace(filtered_map)     # apply your map
      .fillna(df_videogamesales["Platform"])  # keep originals if NaN
)

In [ ]:
# what exactly changed?
changed = df_videogamesales.loc[
    df_videogamesales["Platform"] != df_videogamesales["Platform_norm"],
    ["Platform", "Platform_norm"]
].drop_duplicates()

print("Changed pairs:\n", changed)

# leftovers that might still be odd/abbreviated
print("\nUnmapped platforms after normalization:")
print(sorted(set(df_videogamesales["Platform_norm"]) - appears_in_2plus))


Changed pairs:
     Platform      Platform_norm
6         DS        Nintendo DS
15      X360           Xbox 360
16       PS3      PlayStation 3
17       PS2      PlayStation 2
18      SNES     Super Nintendo
25       GBA   Game Boy Advance
32       3DS       Nintendo 3DS
33       PS4      PlayStation 4
101     XOne           Xbox One
107       GC  Nintendo GameCube
713      PSV   PlayStation Vita

Unmapped platforms after normalization:
['2600', 'DC', 'GB', 'GEN', 'GG', 'Game Boy Advance', 'N64', 'NG', 'Nintendo 3DS', 'Nintendo DS', 'Nintendo GameCube', 'PCFX', 'PS', 'PSP', 'SAT', 'SCD', 'Super Nintendo', 'TG16', 'WS', 'WiiU', 'XB']


In [ ]:
df_videogamesales["Platform"] = df_videogamesales["Platform_norm"]
df_videogamesales = df_videogamesales.drop("Platform_norm", axis=1)

display(df_videogamesales.head())

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
df_videogamesales.to_csv("df_videogamesales.csv",index=False)

In [ ]:
# Compare the shapes of the dataframes
print("Shapes of dataframes loaded from XML and CSV:")
print(f"df_videogamesales_xml shape: {df_videogamesales_xml.shape}")
print(f"df_videogamesales shape: {df_videogamesales.shape}")
if df_videogamesales_xml.shape == df_videogamesales.shape:
    print("These datasets are the same. 🚀")

print(f"df_metacritic_xml shape: {df_metacritic_xml.shape}")
print(f"df_metacritic shape: {df_metacritic.shape}")
if df_metacritic_xml.shape == df_metacritic.shape:
    print("These datasets are the same. 🚀")

print(f"df_playtime_xml shape: {df_playtime_xml.shape}")
print(f"df_playtime shape: {df_playtime.shape}")
if df_playtime_xml.shape == df_playtime.shape:
    print("These datasets are the same. 🚀")

# Compare the first few rows of the dataframes
print("\nFirst few rows of df_videogamesales_xml:")
display(df_videogamesales_xml.head(1))
print("\nFirst few rows of df_videogamesales:")
display(df_videogamesales.head(1))

print("\nFirst few rows of df_metacritic_xml:")
display(df_metacritic_xml.head(1))
print("\nFirst few rows of df_metacritic:")
display(df_metacritic.head(1))

print("\nFirst few rows of df_playtime_xml:")
display(df_playtime_xml.head(1))
print("\nFirst few rows of df_playtime:")
display(df_playtime.head(1))

Shapes of dataframes loaded from XML and CSV:
df_videogamesales_xml shape: (16598, 10)
df_videogamesales shape: (16598, 10)
These datasets are the same. 🚀
df_metacritic_xml shape: (14666, 8)
df_metacritic shape: (14666, 8)
These datasets are the same. 🚀
df_playtime_xml shape: (97231, 9)
df_playtime shape: (97231, 9)
These datasets are the same. 🚀

First few rows of df_videogamesales_xml:


,title,platform,release_year,publisher,genres,na_sales_mil,eu_sales_mil,jp_sales_mil,other_sales_mil,global_sales_mil
0,Wii Sports,Wii,2006.0,Nintendo,[Sports],41.49,29.02,3.77,8.46,82.74



First few rows of df_videogamesales:


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74



First few rows of df_metacritic_xml:


...

,title,release,platform,critic_score,user_score,developer,genre(s),rating
1,Portal 2,2011,Xbox 360,95.0,8.8,Valve Software,"[Action, Shooter, Shooter, First-Person, Sci-Fi, Sci-Fi, Arcade]",E10+



First few rows of df_playtime_xml:


,title,platform,release_year,developer,publisher,genres,main_story_hour,main_plus_sides_hour,completionist_hour
0,688(I) Hunter/Killer,PC,1997.0,Sonalysts,Electronic Arts,[Simulation],10.62,35.37,15.83



First few rows of df_playtime:


,name,platform,genres,developer,publisher,release_date,main_story,main_plus_sides,completionist
0,688(I) Hunter/Killer,PC,[Simulation],Sonalysts,Electronic Arts,1997,10.62,35.37,15.83


In [ ]:
print("Percentage of NaN values in df_videogamesales:")
display(df_videogamesales.isnull().sum() / len(df_videogamesales) * 100)

print("\nPercentage of NaN values in df_metacritic:")
display(df_metacritic.isnull().sum() / len(df_metacritic) * 100)

print("\nPercentage of NaN values in df_playtime:")
display(df_playtime.isnull().sum() / len(df_playtime) * 100)

Percentage of NaN values in df_videogamesales:


,0
Name,0.000000
Platform,0.000000
Year,1.632727
Genre,0.000000
Publisher,0.349440
NA_Sales,0.000000
EU_Sales,0.000000
JP_Sales,0.000000
Other_Sales,0.000000
Global_Sales,0.000000



Percentage of NaN values in df_metacritic:


,0
title,0.000000
release,0.000000
platform,0.000000
critic_score,0.047729
user_score,0.054548
developer,0.000000
genre(s),0.000000
rating,13.977908



Percentage of NaN values in df_playtime:


,0
name,0.000000
platform,1.602630
genres,9.283002
developer,11.369483
publisher,14.752358
release_date,13.460047
main_story,18.498632
main_plus_sides,45.467723
completionist,36.137765


In [ ]:
print("Info for df_videogamesales:")
df_videogamesales.info()

print("\nInfo for df_metacritic:")
df_metacritic.info()

print("\nInfo for df_playtime:")
df_playtime.info()

Info for df_videogamesales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16598 non-null  object 
 1   Platform      16598 non-null  object 
 2   Year          16327 non-null  float64
 3   Genre         16598 non-null  object 
 4   Publisher     16540 non-null  object 
 5   NA_Sales      16598 non-null  float64
 6   EU_Sales      16598 non-null  float64
 7   JP_Sales      16598 non-null  float64
 8   Other_Sales   16598 non-null  float64
 9   Global_Sales  16598 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.3+ MB

Info for df_metacritic:
<class 'pandas.core.frame.DataFrame'>
Index: 14666 entries, 1 to 18660
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         14666 non-null  object 
 1   release       14666 non-null  obj

## Meeting the requirements

> Task

Check if the combined video game datasets contain at least 2,500 unique entities and if at least 1,000 entities are present in at least two datasets, using 'Name', 'alias', or 'Title' columns for matching.

### Standardize game names

### Subtask:
Clean and standardize the game names across all three dataframes to ensure consistent matching. This may involve converting to lowercase, removing special characters, and handling variations in naming.


**Reasoning**:
Define a function to clean the game names and apply it to the relevant columns in each dataframe to create a new 'cleaned_name' column.



In [ ]:
df_videogamesales_copy = df_videogamesales.copy()
df_metacritic_copy = df_metacritic.copy()
df_playtime_copy = df_playtime.copy()

In [ ]:
import re

def clean_game_name(name):
    if isinstance(name, str):
        name = name.lower()
        name = re.sub(r'[^\w\s]', '', name) # Remove punctuation and special characters
        name = re.sub(r'\s+', '_', name).strip() # Replace multiple spaces with single space and strip whitespace
        # Add more cleaning steps as needed based on data exploration (e.g., removing subtitles, edition info)
        return name
    return None # Handle non-string values

df_videogamesales_copy['cleaned_name'] = df_videogamesales_copy['Name'].apply(clean_game_name)
df_metacritic_copy['cleaned_name'] = df_metacritic_copy['title'].apply(clean_game_name)
df_playtime_copy['cleaned_name'] = df_playtime_copy['name'].apply(clean_game_name)

display(df_videogamesales_copy[['Name', 'cleaned_name']].head(1))
display(df_metacritic_copy[['title', 'cleaned_name']].head(1))
display(df_playtime_copy[['name', 'cleaned_name']].head(1))

,Name,cleaned_name
0,Wii Sports,wii_sports


,title,cleaned_name
1,Portal 2,portal_2


,name,cleaned_name
0,688(I) Hunter/Killer,688i_hunterkiller


In [ ]:
display(df_videogamesales_copy.cleaned_name.nunique())
display(df_metacritic_copy.cleaned_name.nunique())
display(df_playtime_copy.cleaned_name.nunique())

11468

9864

48919

In [ ]:
cleaned_names_videogamesales = set(df_videogamesales_copy['cleaned_name'].dropna())
cleaned_names_metacritic = set(df_metacritic_copy['cleaned_name'].dropna())
cleaned_names_playtime = set(df_playtime_copy['cleaned_name'].dropna())

# Find games present in at least two datasets
in_videogamesales_and_metacritic = cleaned_names_videogamesales.intersection(cleaned_names_metacritic)
in_videogamesales_and_playtime = cleaned_names_videogamesales.intersection(cleaned_names_playtime)
in_metacritic_and_playtime = cleaned_names_metacritic.intersection(cleaned_names_playtime)

# Combine all intersections and count unique games
all_intersections = in_videogamesales_and_metacritic.union(in_videogamesales_and_playtime, in_metacritic_and_playtime)

print(f"Number of games appearing in at least two datasets: {len(all_intersections)}")

Number of games appearing in at least two datasets: 10985


In [ ]:
# Replace 'Name' with 'cleaned_name' in df_videogamesales_copy
name_col_index_vgs = df_videogamesales_copy.columns.get_loc('Name')
df_videogamesales_copy = df_videogamesales_copy.drop('Name', axis=1)
df_videogamesales_copy.insert(name_col_index_vgs, 'cleaned_name', df_videogamesales_copy.pop('cleaned_name'))

# Replace 'title' with 'cleaned_name' in df_metacritic_copy
name_col_index_mc = df_metacritic_copy.columns.get_loc('title')
df_metacritic_copy = df_metacritic_copy.drop('title', axis=1)
df_metacritic_copy.insert(name_col_index_mc, 'cleaned_name', df_metacritic_copy.pop('cleaned_name'))

# Replace 'name' with 'cleaned_name' in df_playtime_copy
name_col_index_pt = df_playtime_copy.columns.get_loc('name')
df_playtime_copy = df_playtime_copy.drop('name', axis=1)
df_playtime_copy.insert(name_col_index_pt, 'cleaned_name', df_playtime_copy.pop('cleaned_name'))

print("df_videogamesales_copy after replacement:")
display(df_videogamesales_copy.head(2))

print("\ndf_metacritic_copy after replacement:")
display(df_metacritic_copy.head(2))

print("\ndf_playtime_copy after replacement:")
display(df_playtime_copy.head(2))

df_videogamesales_copy after replacement:


,cleaned_name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,wii_sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,super_mario_bros,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24



df_metacritic_copy after replacement:


,cleaned_name,release,platform,critic_score,user_score,developer,genre(s),rating
1,portal_2,2011-04-19,Xbox 360,95.0,8.8,Valve Software,"Action,Shooter,Shooter,First-Person,Sci-Fi,Sci-Fi,Arcade",E10+
2,metal_gear_solid_v_the_phantom_pain,2015-09-01,Xbox One,95.0,7.5,Konami,"Modern,Modern,Action Adventure,Open-World",M



df_playtime_copy after replacement:


,cleaned_name,platform,genres,developer,publisher,release_date,main_story,main_plus_sides,completionist
0,688i_hunterkiller,PC,Simulation,Sonalysts,Electronic Arts,1997-07-04,10.62,35.37,15.83
1,splosion_man,Xbox 360,Action,Twisted Pixel Games,Microsoft Games Studios,2009-07-22,7.60,9.23,18.77


In [ ]:
# Name + Platform + Release Year might be used as an identifier
# We should decided which columns to use from the third dataset. There are way too many information there. Need to dilute.
# Check platforms Nintendo DS -> DS
# Datasets meet all the requirements ✅

In [ ]:
display(df_videogamesales_copy[df_videogamesales_copy["cleaned_name"]==list(all_intersections)[1231]])
display(df_metacritic_copy[df_metacritic_copy["cleaned_name"]==list(all_intersections)[1231]])
display(df_playtime_copy[df_playtime_copy["cleaned_name"]==list(all_intersections)[1231]])

,cleaned_name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
10462,the_whispered_world,PC,2009.0,Adventure,Deep Silver,0.0,0.08,0.0,0.02,0.1


,cleaned_name,release,platform,critic_score,user_score,developer,genre(s),rating
7567,the_whispered_world,2010-04-26,PC,70.0,7.4,Viva Media,"Adventure,General,General,Point-and-Click",E


,cleaned_name,platform,genres,developer,publisher,release_date,main_story,main_plus_sides,completionist
8283,the_whispered_world,"Linux, Mac, PC",Adventure,Daedalic Entertainment,Deep Silver,2010-04-23,10.17,13.07,11.48


In [ ]:
# Here we wanted to see the number of rows we would end up after integrating datasets on a high-level
joint_df = df_videogamesales_copy.merge(df_metacritic_copy, on='cleaned_name', how='inner').merge(df_playtime_copy, on='cleaned_name', how='inner')

In [ ]:
joint_df.sample()

,cleaned_name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,release,platform_x,critic_score,user_score,developer_x,genre(s),rating,platform_y,genres,developer_y,publisher,release_date,main_story,main_plus_sides,completionist
2802,marvel_nemesis_rise_of_the_imperfects,PS2,2005.0,Fighting,Electronic Arts,0.64,0.02,0.0,0.1,0.77,2005-09-20,PlayStation 2,53.0,6.6,EA Games,"Action,Fighting,3D",T,"Nintendo DS, Nintendo GameCube, PlayStation 2, PlayStation Portable, Xbox",Fighting,Nihilistic Software,Electronic Arts,2005-09-20,4.43,6.02,7.0


In [ ]:
display(joint_df.shape)
display(joint_df.info())

(10746, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10746 entries, 0 to 10745
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cleaned_name     10746 non-null  object 
 1   Platform         10746 non-null  object 
 2   Year             10525 non-null  float64
 3   Genre            10746 non-null  object 
 4   Publisher        10740 non-null  object 
 5   NA_Sales         10746 non-null  float64
 6   EU_Sales         10746 non-null  float64
 7   JP_Sales         10746 non-null  float64
 8   Other_Sales      10746 non-null  float64
 9   Global_Sales     10746 non-null  float64
 10  release          10746 non-null  object 
 11  platform_x       10746 non-null  object 
 12  critic_score     10744 non-null  float64
 13  user_score       10744 non-null  object 
 14  developer_x      10746 non-null  object 
 15  genre(s)         10746 non-null  object 
 16  rating           10654 non-null  object 
 17  platform_y  

None

In [ ]:
joint_df.shape[0]
# This is the roughly the number for requirement 2. We'll get more or less a number of rows close to this in the final integrated dataset.
# It's way more than set out 2,500 entities and almost at the edge of desirable 10,000 entities.

10746

In [ ]:
df_playtime.to_csv("df_playtime.csv",index=False)
df_metacritic.to_csv("df_metacritic.csv",index=False)
df_videogamesales.to_csv("df_videogamesales.csv",index=False)

# Steam (Extra)
> **If the current three datasets are found inadequate, we can also merge this one to obtain additional information or address more complex issues.**

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/FronkonGames/steam-games-dataset/data/train-00000-of-00001-e2ed184370a06932.parquet")
#https://huggingface.co/datasets/FronkonGames/steam-games-dataset

In [ ]:
df.shape

(83560, 39)

In [ ]:
df[df["Name"].str.contains("Red Dead Redemption",na=False,case=False)].head(1)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
43468,1174180,Red Dead Redemption 2,"Dec 5, 2019",5000000 - 10000000,26520,17,29.99,0,"America, 1899. Arthur Morgan and the Van der Linde gang are outlaws on the run. With federal agents and the best bounty hunters in the nation mass...","['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Simplified Chinese', 'Span...","['English', 'Traditional Chinese']",None,https://cdn.akamai.steamstatic.com/steam/apps/1174180/header.jpg?t=1656615305,https://www.rockstargames.com/reddeadredemption2/,https://support.rockstargames.com/,None,True,False,False,93,https://www.metacritic.com/game/pc/red-dead-redemption-2?ftag=MCD-06-10aaa1f,0,251841,32592,NaN,51,256117,None,5776,619,1811,669,Rockstar Games,Rockstar Games,"Single-player,Multi-player,PvP,Online PvP,Co-op,Online Co-op,Steam Achievements,In-App Purchases,Partial Controller Support,Remote Play on Phone,R...","Action,Adventure","Open World,Adventure,Story Rich,Western,Action,Multiplayer,Realistic,Shooter,Singleplayer,Atmospheric,Third-Person Shooter,Horses,Beautiful,Mature...","https://cdn.akamai.steamstatic.com/steam/apps/1174180/ss_66b553f4c209476d3e4ce25fa4714002cc914c4f.1920x1080.jpg?t=1656615305,https://cdn.akamai.st...","http://cdn.akamai.steamstatic.com/steam/apps/256767979/movie_max.mp4?t=1574881964,http://cdn.akamai.steamstatic.com/steam/apps/256767971/movie_max..."


In [ ]:
df_steam_copy = df.copy()

# Normalize the 'Name' column in df_steam_copy and replace it
df_steam_copy['cleaned_name'] = df_steam_copy['Name'].apply(clean_game_name)
name_col_index_steam = df_steam_copy.columns.get_loc('Name')
df_steam_copy = df_steam_copy.drop('Name', axis=1)
df_steam_copy.insert(name_col_index_steam, 'cleaned_name', df_steam_copy.pop('cleaned_name'))

# Get the cleaned names from the steam dataset
cleaned_names_steam = set(df_steam_copy['cleaned_name'].dropna())

# Find games present in at least two datasets, including the steam dataset
all_cleaned_names = cleaned_names_videogamesales.union(cleaned_names_metacritic, cleaned_names_playtime, cleaned_names_steam)

in_videogamesales_and_steam = cleaned_names_videogamesales.intersection(cleaned_names_steam)
in_metacritic_and_steam = cleaned_names_metacritic.intersection(cleaned_names_steam)
in_playtime_and_steam = cleaned_names_playtime.intersection(cleaned_names_steam)

all_intersections_with_steam = all_intersections.union(in_videogamesales_and_steam, in_metacritic_and_steam, in_playtime_and_steam)

print(f"Number of unique games across all datasets: {len(all_cleaned_names)}")
print(f"Number of games appearing in at least two datasets (including Steam): {len(all_intersections_with_steam)}")

Number of unique games across all datasets: 120373
Number of games appearing in at least two datasets (including Steam): 28709


In [ ]:
display(len(in_videogamesales_and_steam))
display(len(in_metacritic_and_steam))
display(len(in_playtime_and_steam))

967

4045

20491

In [ ]:
len(cleaned_names_videogamesales)

11468

In [ ]:
df_steam_copy.head(1)

,AppID,cleaned_name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,galactic_bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-playe...,['English'],[],None,https://cdn.akamai.steamstatic.com/steam/apps/20200/header.jpg?t=1640121033,http://www.galacticbowling.net,None,None,True,False,False,0,None,0,6,11,NaN,30,0,None,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,Partial Controller Support","Casual,Indie,Sports","Indie,Casual,Sports,Bowling","https://cdn.akamai.steamstatic.com/steam/apps/20200/0000005994.1920x1080.jpg?t=1640121033,https://cdn.akamai.steamstatic.com/steam/apps/20200/0000...",http://cdn.akamai.steamstatic.com/steam/apps/256863704/movie_max.mp4?t=1638854607


In [ ]:
super_joint_df=joint_df.merge(df_steam_copy, on='cleaned_name', how='inner')

In [ ]:
display(super_joint_df.shape)
display(super_joint_df.info())

(3932, 81)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3932 entries, 0 to 3931
Data columns (total 81 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rank                        3932 non-null   int64  
 1   cleaned_name                3932 non-null   object 
 2   Platform                    3932 non-null   object 
 3   Year                        3829 non-null   float64
 4   Genre                       3932 non-null   object 
 5   Publisher                   3926 non-null   object 
 6   NA_Sales                    3932 non-null   float64
 7   EU_Sales                    3932 non-null   float64
 8   JP_Sales                    3932 non-null   float64
 9   Other_Sales                 3932 non-null   float64
 10  Global_Sales                3932 non-null   float64
 11  release                     3932 non-null   object 
 12  platform_x                  3932 non-null   object 
 13  critic_score                3926 

None

In [ ]:
super_joint_df

,Rank,cleaned_name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,release,platform_x,critic_score,user_score,developer_x,genre(s),rating,id,type,platform_y,genres,developer_y,publisher,release_date,release_precision,release_year,release_month,release_day,main_story_polled,main_story,main_plus_sides_polled,main_plus_sides,completionist_polled,completionist,all_styles_polled,all_styles,single_player_polled,single_player,co_op_polled,co_op,versus_polled,versus,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,17,grand_theft_auto_v,PS3,2013.0,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40,2015-04-13,PC,96.0,7.8,Rockstar Games,"Modern,Action Adventure,Open-World",M,4064,game,"PC, PlayStation 3, PlayStation 4, PlayStation 5, Xbox 360, Xbox One, Xbox Series X/S","Action, Driving, First-Person, Open World, Racing, Shooter, Third-Person",Rockstar North,Rockstar Games,2013-09-17,day,2013.0,9.0,17.0,3.0,32.60,3.0,54.67,908.0,94.45,7.0,50.38,3.0,32.60,347.0,287.0,233.0,205.0,271590,"Apr 13, 2015",20000000 - 50000000,170527,17,0.00,1,"When a young street hustler, a retired bank robber and a terrifying psychopath find themselves entangled with some of the most frightening and der...","['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Korean', 'Polish', 'Portuguese - Brazil', 'Russian', 'Traditional Chinese', 'Japane...","['English', 'Spanish - Latin America']",None,https://cdn.akamai.steamstatic.com/steam/apps/271590/header.jpg?t=1618856444,http://www.rockstargames.com/V/,https://support.rockstargames.com,None,True,False,False,96,https://www.metacritic.com/game/pc/grand-theft-auto-v?ftag=MCD-06-10aaa1f,0,1171197,210154,NaN,77,1247051,None,13650,656,6274,154,Rockstar North,Rockstar Games,"Single-player,Multi-player,PvP,Online PvP,Co-op,Online Co-op,Steam Achievements,Full controller support,Remote Play on Phone,Remote Play on Tablet...","Action,Adventure","Open World,Action,Multiplayer,Automobile Sim,Crime,Third Person,First-Person,Shooter,Mature,Adventure,Singleplayer,Third-Person Shooter,Racing,Co-...","https://cdn.akamai.steamstatic.com/steam/apps/271590/ss_32aa18ab3175e3002217862dd5917646d298ab6b.1920x1080.jpg?t=1618856444,https://cdn.akamai.ste...","http://cdn.akamai.steamstatic.com/steam/apps/256738416/movie_max.mp4?t=1563930911,http://cdn.akamai.steamstatic.com/steam/apps/256738408/movie_max..."
1,17,grand_theft_auto_v,PS3,2013.0,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40,2014-11-18,PlayStation 4,97.0,8.4,Rockstar Games,"Action Adventure,Modern,Open-World",M,4064,game,"PC, PlayStation 3, PlayStation 4, PlayStation 5, Xbox 360, Xbox One, Xbox Series X/S","Action, Driving, First-Person, Open World, Racing, Shooter, Third-Person",Rockstar North,Rockstar Games,2013-09-17,day,2013.0,9.0,17.0,3.0,32.60,3.0,54.67,908.0,94.45,7.0,50.38,3.0,32.60,347.0,287.0,233.0,205.0,271590,"Apr 13, 2015",20000000 - 50000000,170527,17,0.00,1,"When a young street hustler, a retired bank robber and a terrifying psychopath find themselves entangled with some of the most frightening and der...","['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Korean', 'Polish', 'Portuguese - Brazil', 'Russian', 'Traditional Chinese', 'Japane...","['English', 'Spanish - Latin America']",None,https://cdn.akamai.steamstatic.com/steam/apps/271590/header.jpg?t=1618856444,http://www.rockstargames.com/V/,https://support.rockstargames.com,None,True,False,False,96,https://www.metacritic.com/game/pc/grand-theft-auto-v?ftag=MCD-06-10aaa1f,0,1171197,210154,NaN,77,1247051,None,13650,656,6274